In [1]:
import pandas as pd
import numpy as np

In [2]:
import pyomo.environ as pyo
import pyomo.opt as pyopt

from copt_pyomo import *

Cardinal Optimizer v5.0.4. Build date Aug 19 2022
Copyright Cardinal Operations 2022. All Rights Reserved



In [3]:
class Data:
    def __init__(self, path: str) -> None:    
        demand_path = path + "/forecast.csv"
        existingEV_path = path + "/existing_EV_infrastructure_2018.csv"
        self.df_dem = pd.read_csv(demand_path)
        self.df_dem = self.df_dem.set_index("demand_point_index", drop=True)
        self.df_ex = pd.read_csv(existingEV_path)
        self.df_ex = self.df_ex.set_index("supply_point_index", drop=True)
        
path = "data"
data = Data(path)

In [4]:
data.df_ex

,x_coordinate,y_coordinate,total_parking_slots,existing_num_SCS,existing_num_FCS
supply_point_index,,,,,
0,50.163110,19.412014,23,5,3
1,37.336451,58.119225,27,4,7
2,46.709232,57.525650,31,6,14
3,30.528626,55.379835,26,5,5
4,51.521781,35.116755,32,11,6
...,...,...,...,...,...
95,45.471204,20.999414,24,3,4
96,30.318396,33.388335,32,5,10
97,36.218839,22.235766,32,4,14


In [102]:
from distutils.command.install_scripts import install_scripts
from inspect import BoundArguments


class OptModel(pyo.ConcreteModel):
    def __init__(self, data):
        super().__init__()
        self.data = data
    
    def buildSets(self):
        self.i = pyo.Set(initialize=self.data.df_dem.index)
        self.j = pyo.Set(initialize=self.data.df_ex.index)
        self.k = pyo.Set(initialize=[2019, 2020])
        self.solver = pyopt.SolverFactory('copt_direct')
        # self.solver.options["relgap"] = 0.1
    
    def buildParams(self):
        def d_init(self, i, k):
            return self.data.df_dem.loc[i, f"{k}"]
        self.d = pyo.Param(self.i, self.k, initialize=d_init)
        
        def exSCS_init(self, j):
            return self.data.df_ex.loc[j, "existing_num_SCS"]
        self.exSCS = pyo.Param(self.j, initialize=exSCS_init)
        
        def exFCS_init(self, j):
            return self.data.df_ex.loc[j, "existing_num_FCS"]
        self.exFCS = pyo.Param(self.j, initialize=exFCS_init)
        
        def numPS_init(self, j):
            return self.data.df_ex.loc[j, "total_parking_slots"]
        self.numPS = pyo.Param(self.j, initialize=numPS_init)
        
        def dist_init(self, i ,j):
            return np.sqrt((self.data.df_ex.loc[j, "x_coordinate"] - self.data.df_dem.loc[i, "x_coordinate"]) ** 2.0 +
                           (self.data.df_ex.loc[j, "y_coordinate"] - self.data.df_dem.loc[i, "y_coordinate"]) ** 2.0)
        self.dist = pyo.Param(self.i, self.j, initialize=dist_init)
        
        self.capSCS = 200
        self.capFCS = 400
        
    def buildVars(self):
        self.ds = pyo.Var(self.i, self.j, self.k, domain=pyo.NonNegativeReals)
        self.insSCS = pyo.Var(self.j, self.k, domain=pyo.NonNegativeIntegers)
        self.insFCS = pyo.Var(self.j, self.k, domain=pyo.NonNegativeIntegers)

    def buildConstraints(self):
        def satDemCtr_rule(self, j, k):
            if k == 2019:
                existing_SCS = self.exSCS[j] + self.insSCS[j, 2019]
                existing_FCS = self.exFCS[j] + self.insFCS[j, 2019]
            else:
                existing_SCS = self.exSCS[j] + pyo.quicksum(self.insSCS[j, k1] for k1 in self.k)
                existing_FCS = self.exFCS[j] + pyo.quicksum(self.insFCS[j, k1] for k1 in self.k)
            ret = pyo.quicksum(self.ds[i, j, k] for i in self.i) <= \
                self.capSCS * existing_SCS + self.capFCS * existing_FCS
            return ret 
        self.satDemCtr = pyo.Constraint(self.j, self.k, rule=satDemCtr_rule)
        
        def forDemCtr_rule(self, i, k):
            ret = pyo.quicksum(self.ds[i, j, k] for j in self.j) == \
                self.d[i, k]
            return ret 
        self.forDemCtr = pyo.Constraint(self.i, self.k, rule=forDemCtr_rule) 
        
        def maxStaCtr_rule(self, j, k):
            if self.numPS[j] == 0:
                pyo.Constraint.Skip 
            if k == 2019:
                existing_SCS = self.exSCS[j] + self.insSCS[j, 2019]
                existing_FCS = self.exFCS[j] + self.insFCS[j, 2019]
            else:
                existing_SCS = self.exSCS[j] + pyo.quicksum(self.insSCS[j, k1] for k1 in self.k)
                existing_FCS = self.exFCS[j] + pyo.quicksum(self.insFCS[j, k1] for k1 in self.k)
            ret = existing_FCS + existing_SCS <= self.numPS[j]
            return ret 
        self.maxStaCtr = pyo.Constraint(self.j, self.k, rule=maxStaCtr_rule)
        
    def buildObjective(self):
        def objRule(self):
            return pyo.quicksum(self.dist[i, j] * self.ds[i, j, k] for k in self.k for j in self.j for i in self.i) \
                + 600 * pyo.quicksum(self.insSCS[j, k] + 1.5 * self.insFCS[j, k] for k in self.k for j in self.j)
        self.cost = pyo.Objective(rule=objRule, sense=pyo.minimize)
        
    def solve(self):
        self.results = self.solver.solve(self, tee=True) 
    
    def resultsToDf(self):
        years = [2019, 2020]
        df_insSCS = pd.DataFrame(columns=["supply_point_index"])
        df_insSCS = df_insSCS.set_index("supply_point_index")
        df_insFCS = pd.DataFrame(columns=["supply_point_index"])
        df_insFCS = df_insFCS.set_index("supply_point_index")
        df_ds = dict()
        for k in self.k:
            df_ds[k] = pd.DataFrame(columns=["demand_point_index"])
            df_ds[k] = df_ds[k].set_index("demand_point_index")
            for j in self.j:
                df_insSCS.loc[j, k] = self.insSCS[j, k].value   
                df_insFCS.loc[j, k] = self.insFCS[j, k].value              
                for i in self.i:
                    df_ds[k].loc[i, j] = self.ds[i, j, k].value
            df_ds[k].to_csv(f"ds_{k}.csv")   
            
        df_insSCS.to_csv("insSCS.csv")
        df_insFCS.to_csv("insFCS.csv")
        

In [103]:
model = OptModel(data)
model.buildSets()
model.buildParams()
model.buildVars()
model.buildConstraints()
model.buildObjective()

In [104]:
# model.maxStaCtr.pprint()

In [105]:
model.solve()

Model fingerprint: cc4de685

Hardware has 10 cores and 10 threads. Using instruction set ARMV8 (30)
Minimizing a MIP problem

The original problem has:
    8592 rows, 819600 columns and 1639600 non-zero elements
    400 integers

Presolving the problem

The presolved problem has:
    8591 rows, 819598 columns and 1639594 non-zero elements
    2 binaries and 396 integers

Starting the MIP solver with 10 threads and 32 tasks

     Nodes    Active  LPit/n  IntInf     BestBound  BestSolution    Gap   Time
         0         1      --       0  0.000000e+00            --    Inf  7.35s
         0         1      --      25  3.372405e+06            --    Inf 13.86s
H        0         1      --      25  3.372405e+06  3.382245e+06  0.29% 13.87s
         0         1      --      25  3.372405e+06  3.382245e+06  0.29% 18.77s
         0         1      --      18  3.372489e+06  3.382245e+06  0.29% 20.20s
H        0         1      --      18  3.372489e+06  3.378981e+06  0.19% 20.22s
         0         

In [ ]:
model.resultsToDf()

In [26]:
# model.write('model.lp', io_options={'symbolic_solver_labels':True})

('model.lp', 10992564832)

In [14]:
# from pyomo.util.infeasible import log_infeasible_constraints
# import logging

In [15]:
# log_infeasible_constraints(model, log_expression=True, log_variables=True)
# logging.basicConfig(filename='infeas.log', encoding='utf-8', level=logging.INFO)
# # print(value(m.z))